# Saving the values of period ratios from 1.1-1.5 from the NASA Exoplanet Archive to a csv
We want to save planets that are in trios that have an inner compact period ratio (1.1-1.5 since those below 1.1 should be unstable) to match our simulations for multiplanet (>3) systems to a csv to use for our other notebooks! A trio is a group of three adjacent planets in a system. The period ratio between the outer period ratios can be anything as long as the inner is compact.

In [2]:
import pandas as pd
from tqdm import tqdm

In [4]:
#MAKE INITIAL DATA FRAME

df = pd.read_csv('20230302_all_exoplanets.csv')

# Create an array of all host star/system names
hosts = df['hostname'].unique()

# For each planetary system...
for host in tqdm(hosts):
    # If the system contains less than 3 planets...
    if (df['hostname'] == host).sum() < 3:
        # Get the row indices of the system's planets
        index = df[df['hostname']==host].index
        # Remove those rows from our DataFrame
        df.drop(index, inplace=True)

# Data that might be useful to us
planet_data = pd.DataFrame()
planet_data["Planet Name"] = df['pl_name']
planet_data["Host Name"] = df['hostname']
planet_data["Orbital Period"] = df['pl_orbper']
planet_data["Discovery Method"] = df['discoverymethod']
planet_data["Semi-Major Axis (AU)"] = df['pl_orbsmax']
planet_data["Eccentricity"] = df['pl_orbeccen']
planet_data["Radius (R_earth)"] = df['pl_rade']
planet_data["Mass or Mass * sin(i) (M_earth)"] = df['pl_bmasse']
planet_data["Stellar Mass (M_sun)"] = df['st_mass']
planet_data.to_csv('planet_data.csv')
df = pd.read_csv('planet_data.csv', index_col=0)
df.head()

100%|█████████████████████████████████████████████████████| 3943/3943 [00:08<00:00, 482.92it/s]


,Planet Name,Host Name,Orbital Period,Discovery Method,Semi-Major Axis (AU),Eccentricity,Radius (R_earth),Mass or Mass * sin(i) (M_earth),Stellar Mass (M_sun)
17,2MASS J19383260+4603591 b,2MASS J19383260+4603591,406.0000,Eclipse Timing Variations,0.9200,0.330,13.4,603.8770,0.48
22,47 UMa b,47 UMa,1078.0000,Radial Velocity,2.1000,0.032,13.2,804.0800,1.06
23,47 UMa c,47 UMa,2391.0000,Radial Velocity,3.6000,0.098,14.2,171.6210,1.06
24,47 UMa d,47 UMa,14002.0000,Radial Velocity,11.6000,0.160,13.5,521.2200,1.06
27,55 Cnc b,55 Cnc,14.6516,Radial Velocity,0.1134,0.000,13.9,263.9785,0.91


In [5]:
df = pd.read_csv('planet_data.csv', index_col=0)
len(df)

1041

In [6]:
# GET RID OF ONES THAT DON'T HAVE INNER PRATIO 1.1-1.5
df['Keep'] = False

# for every system in planet_data
for system in df['Host Name'].unique():
    # new system df of JUST planets from that system
    system_df = df[df['Host Name'] == system]
    # sort them in order of orbital period lowest to highest
    system_sorted = system_df.sort_values(by='Orbital Period')
    
    Nplanets = system_df.shape[0] # number of planets in system
    Ntrios = Nplanets - 2 # number of trios, three adjacent planet groupings
    
    # for every trio
    for q in range(Ntrios):
        P1 = system_sorted.iloc[q]['Orbital Period']
        P2 = system_sorted.iloc[q+1]['Orbital Period']
        P3 = system_sorted.iloc[q+2]['Orbital Period']
        
        # if the inner period ratio is 1.1-1.5
        if P2/P1 > 1.1 and P2/P1 < 1.5:
            # keep these planets in dataframe
            df.loc[system_sorted.iloc[q].name, 'Keep'] = True
            df.loc[system_sorted.iloc[q+1].name, 'Keep'] = True
            df.loc[system_sorted.iloc[q+2].name, 'Keep'] = True
            

In [7]:
# create new data frame with trios that have the inner pratio 1.1-1.5
compact_df = df[df['Keep'] == True]

# get rid of 'keep' column since these would all be true and it's not important anymore
del compact_df['Keep']

print(len(compact_df))

compact_df.to_csv('compact_planet_data.csv')

df = pd.read_csv('compact_planet_data.csv', index_col=0)
df.head()

161


,Planet Name,Host Name,Orbital Period,Discovery Method,Semi-Major Axis (AU),Eccentricity,Radius (R_earth),Mass or Mass * sin(i) (M_earth),Stellar Mass (M_sun)
119,DMPP-1 b,DMPP-1,18.570,Radial Velocity,0.1462,0.083,5.29,24.27,1.21
120,DMPP-1 c,DMPP-1,6.584,Radial Velocity,0.0733,0.057,3.06,9.60,1.21
122,DMPP-1 e,DMPP-1,5.516,Radial Velocity,0.0651,0.070,1.86,4.13,1.21
194,GJ 180 b,GJ 180,17.133,Radial Velocity,0.0920,0.070,2.43,6.49,0.43
195,GJ 180 c,GJ 180,24.329,Radial Velocity,0.1290,0.090,2.41,6.40,0.43


In [4]:
df = pd.read_csv('compact_planet_data.csv', index_col=0)
len(df)

161